In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from llamabot import SimpleBot


In [ ]:
# bot = SimpleBot("You are Richard Feynman", model_name="orca-mini", verbose=True)
# bot("Tell me about black holes.")


Yes, looks like the thing works with both verbose = True and verbose = False!


In [ ]:
# try writing commit diff
from llamabot import SimpleBot
from llamabot.prompt_manager import prompt

bot = SimpleBot(
    "You are an expert user of Git.",
    # model_name=os.getenv("OPENAI_DEFAULT_MODEL", "gpt-3.5-turbo-16k-0613"),
    model_name="codellama:13b",
)


In [ ]:
diff = 'diff --git a/llamabot/bot/model_dispatcher.py b/llamabot/bot/model_dispatcher.py\nindex eab00d5..abe4d70 100644\n--- a/llamabot/bot/model_dispatcher.py\n+++ b/llamabot/bot/model_dispatcher.py\n@@ -12,6 +12,7 \n@@ from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler\n from langchain.callbacks.base import BaseCallbackManager\n from time import sleep\n from loguru import logger\n+from functools import partial\n \n # get this list from: https://ollama.ai/library\n ollama_model_keywords = [\n@@ -62,13 +63,14 @@ def create_model(\n     :param verbose: (LangChain config) Whether to print debug messages.\n     :return: The model.\n     """\n-    ModelClass = ChatOpenAI\n+    # We use a `partial` here to ensure that we have the correct way of specifying\n+    # a model name between ChatOpenAI and ChatOllama.\n+    ModelClass = partial(ChatOpenAI, model_name=model_name)\n     if model_name.split(":")[0] in ollama_model_keywords:\n-        ModelClass = ChatOllama\n+        ModelClass = partial(ChatOllama, model=model_name)\n         launch_ollama(model_name, verbose=verbose)\n \n     return ModelClass(\n-        model_name=model_name,\n         temperature=temperature,\n         streaming=streaming,\n         verbose=verbose,'


In [ ]:
# from llamabot.code_manipulation import get_git_diff
# from pyprojroot import here

# diff = get_git_diff(here())

# print(diff)


In [ ]:
@prompt
def compose_commit_message(diff, template):
    """Here is a git diff:

    {{ diff }}

    Here is the template for writing commit messages:

    {{ template }}

    Please compose a commit message for the provided diff in the format of the template.
    Return only the commit message and nothing else.

    [AI]: Here is a possible commit message for the provided diff:
    """


template = """
<type>[optional scope]: <description>

[body describing in detail the changes made to the code]

[optional footer(s)]
"""


bot(compose_commit_message(diff, template))


Ok, so I have new documented behaviour.

If:

1. The Ollama model server is running (e.g. Ollama app on macOS),
2. The Ollama model has been downloaded,

Then:

1. LangChain will autolaunch the Ollama model API server (I think?)
2. I don't need to magically launch the process within LlamaBot.

Ok, that's neat, and I think I like this better.

In [ ]:
"""Dispatch"""
import contextvars

from langchain.callbacks.base import BaseCallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOllama, ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

from llamabot.config import default_language_model
from llamabot.recorder import autorecord

prompt_recorder_var = contextvars.ContextVar("prompt_recorder")


In [ ]:
model_name = "codellama"
temperature = 0.0
verbose = True
streaming = True

llm = ChatOpenAI(
    model_name=model_name,
    temperature=temperature,
    streaming=streaming,
    verbose=verbose,
    callback_manager=BaseCallbackManager(
        handlers=[StreamingStdOutCallbackHandler()] if streaming else []
    ),
)

llm()


In [ ]:
def dispatch_model(
    model_name,
    temperature=0.0,
    streaming=True,
    verbose=True,
):
    """Dispatch and create the right model"""
    pass


class SimpleBot:
    """Simple Bot that is primed with a system prompt, accepts a human message,
    and sends back a single response.

    This bot does not retain chat history.
    """

    def __init__(
        self,
        system_prompt,
        temperature=0.0,
        model_name="codellama",
        streaming=True,
        verbose=True,
    ):
        """Initialize the SimpleBot.

        :param system_prompt: The system prompt to use.
        :param temperature: The model temperature to use.
            See https://platform.openai.com/docs/api-reference/completions/create#completions/create-temperature
            for more information.
        :param model_name: The name of the OpenAI model to use.
        :param streaming: (LangChain config) Whether to stream the output to stdout.
        :param verbose: (LangChain config) Whether to print debug messages.
        """
        self.system_prompt = system_prompt
        self.model = ChatOllama(
            model_name=model_name,
            temperature=temperature,
            streaming=streaming,
            verbose=verbose,
            callback_manager=BaseCallbackManager(
                handlers=[StreamingStdOutCallbackHandler()] if streaming else []
            ),
        )
        self.chat_history = []
        self.model_name = model_name

    def __call__(self, human_message: str) -> AIMessage:
        """Call the SimpleBot.

        :param human_message: The human message to use.
        :return: The response to the human message, primed by the system prompt.
        """

        messages = [
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=human_message),
        ]
        response = self.model(messages)
        self.chat_history.append(HumanMessage(content=human_message))
        self.chat_history.append(response)
        autorecord(human_message, response.content)
        return response


In [ ]:
bot = SimpleBot("You are a coding expert", model_name="codellama")


In [ ]:
bot("how do you write a FastAPI endpoint?")
